<a target="_blank" href="https://colab.research.google.com/github/sergiopaniego/RAG_local_tutorial/blob/main/example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Simple RAG example with Langchain, Ollama and and open-source LLM model

Sources:

* https://github.com/svpino/llm
* https://github.com/AIAnytime/Gemma-7B-RAG-using-Ollama/blob/main/Ollama%20Gemma.ipynb
* https://www.youtube.com/watch?v=-MexTC18h20&ab_channel=AIAnytime
* https://www.youtube.com/watch?v=HRvyei7vFSM&ab_channel=Underfitted

 
# Requirements

* Ollama installed locally

# Install the requirements

If an error is raised related to docarray, refer to this solution: https://stackoverflow.com/questions/76880224/error-using-using-docarrayinmemorysearch-in-langchain-could-not-import-docarray

In [ ]:
!pip3 install langchain
!pip3 install langchain_pinecone
!pip3 install langchain[docarray]
!pip3 install docarray
!pip3 install pypdf

# Select the LLM model to use

The model must be downloaded locally to be used, so if you want to run llama3, you should run:

```

ollama pull llama3

```

Check the list of models available for Ollama here: https://ollama.com/library

In [7]:
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "gemma:7b"
#MODEL = "llama2"
MODEL = "llama3" # https://ollama.com/library/llama3

# We instanciate the LLM model and the Embedding model

In [8]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Give me an inspirational quote")

'Here\'s one that I hope will inspire you:\n\n"Believe you can and you\'re halfway there." - Theodore Roosevelt\n\nI hope this quote gives you a boost of motivation and confidence to tackle your goals and challenges!'

In [9]:
model.invoke("Waht is 2+2?")

'The answer to 2+2 is 4.'

## Using a parser provided by LangChain, we can transform the LLM output to something more suitable to be read

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
response_from_model = model.invoke("Give me an inspirational quote")
parsed_response = parser.parse(response_from_model)
print(parsed_response)

Here's one:

"Believe you can and you're halfway there." - Theodore Roosevelt

I hope it inspires you to tackle your goals and pursue your passions!


# We generate the template for the conversation with the instruct-based LLM

We can create a template to structure the conversation effectively.

This template allows us to provide some general context to the Language Learning Model (LLM), which will be utilized for every prompt. This ensures that the model has a consistent background understanding for all interactions.

Additionally, we can include specific context relevant to the particular prompt. This helps the model understand the immediate scenario or topic before addressing the actual question. Following this specific context, we then present the actual question we want the model to answer.

By using this approach, we enhance the model's ability to generate accurate and relevant responses based on both the general and specific contexts provided.

In [11]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, answer with "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, answer with "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

The model can answer prompts based on the context:

In [12]:
formatted_prompt = prompt.format(context="My parents named me Sergio", question="What's your name?")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response)

My name is Sergio!


But it can't answer what is not provided as context:

In [13]:
formatted_prompt = prompt.format(context="My parents named me Sergio", question="What's my age?")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response)

I don't know!


Even previously known info!

In [14]:
formatted_prompt = prompt.format(context="My parents named me Sergio", question="What is 2+2?")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response)

I don't know!


# Load an example PDF to do Retrieval Augmented Generation (RAG)

For the example, you can select your own PDF.

In [15]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("example.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Ser gio Paniego\nI’m a PhD candidate at Univ ersidad Re y Juan Carlos , Madrid, r esear ching ar ti\x00cial intelligence,\ncomputer vision and r obotics and its application t o aut onomous driving. I obtained an\nInformatics Engineering and Softwar e Engineering Dual Degr ee at Univ ersidad Re y Juan\nCarlos and a MSc in Ar ti\x00cial Intelligence at Univ ersidad P olitécnica de Madrid with a José\nCuena awar d.\nThe pr ojects I’ m working on now can be found in my GitHub account .\nIn addition t o resear ch, I’ m also a teaching assistant for some courses (Ar ti\x00cial Intelligence\nand Robotics) for thir d-year students at the same uni r elated t o my r esear ch inter ests. I ha ve\nalso de veloped a course for Platzi  platform called Object Detection and Segmentation with\nTensor\x00ow  among others ( teaching section ). I also lik e to giv e talks  about my work and\narti\x00cial intelligence in gener al.\nI have par ticipated in Google Summer of Code  prog

# Store the PDF in a vector space.

From Langchain docs:

`DocArrayInMemorySearch is a document index provided by Docarray that stores documents in memory. It is a great starting point for small datasets, where you may not want to launch a database server.`

The execution time of the following block depends on the complexity and longitude of the PDF provided. Try to keep it small and simple for the example.

In [16]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

/Users/sergiopaniegoblanco/Library/Python/3.9/lib/python/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


# Create retriever of vectors that are similar to be used as context

In [17]:
retriever = vectorstore.as_retriever()
retriever.invoke("artificial intelligence")

[Document(page_content='Ser gio Paniego\nI’m a PhD candidate at Univ ersidad Re y Juan Carlos , Madrid, r esear ching ar ti\x00cial intelligence,\ncomputer vision and r obotics and its application t o aut onomous driving. I obtained an\nInformatics Engineering and Softwar e Engineering Dual Degr ee at Univ ersidad Re y Juan\nCarlos and a MSc in Ar ti\x00cial Intelligence at Univ ersidad P olitécnica de Madrid with a José\nCuena awar d.\nThe pr ojects I’ m working on now can be found in my GitHub account .\nIn addition t o resear ch, I’ m also a teaching assistant for some courses (Ar ti\x00cial Intelligence\nand Robotics) for thir d-year students at the same uni r elated t o my r esear ch inter ests. I ha ve\nalso de veloped a course for Platzi  platform called Object Detection and Segmentation with\nTensor\x00ow  among others ( teaching section ). I also lik e to giv e talks  about my work and\narti\x00cial intelligence in gener al.\nI have par ticipated in Google Summer of Code  prog

# Generate conversate with the document to extract the details

In [18]:
# Assuming retriever is an instance of a retriever class and has a method to retrieve context
retrieved_context = retriever.invoke("artificial intelligence")

In [19]:
questions = [
    "What are his research interests?",
    "Does he have teaching experience?",
    "What are his hobbies?"
]

for question in questions:
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: What are his research interests?
Answer: Based on the context, Ser gio Paniego's research interests include artificial intelligence, computer vision, and robotics, with a focus on autonomous driving.

Question: Does he have teaching experience?
Answer: Yes, according to the context, Ser gio Paniego has teaching experience as a teaching assistant for some courses (Ar ti\x00cial Intelligence and Robotics) for third-year students at Univ ersidad Re y Juan Carlos. He also developed a course for Platzi platform called Object Detection and Segmentation with Tensor\x00ow among others.

Question: What are his hobbies?
Answer: According to the context, one of Ser gio Paniego's hobbies is photography.



# Loop to ask-answer questions continously

In [20]:
while True:
    print("Say 'exit' or 'quit' to exit the loop")
    question = input('User question: ')
    print(f"Question: {question}")
    if question.lower() in ["exit", "quit"]:
        print("Exiting the conversation. Goodbye!")
        break
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = model.invoke(formatted_prompt)
    parsed_response = parser.parse(response_from_model)
    print(f"Answer: {parsed_response}")
    print()

Say 'exit' or 'quit' to exit the loop
Question: Does he kwon about deep learning?
Answer: Yes. According to the context, Ser gio Paniego has worked on projects related to artificial intelligence, computer vision, and robotics, including Deep Learning Studio with JdeRobot Robotics and Artificial Intelligence association. He also mentions having a course for Platzi platform called Object Detection and Segmentation with TensorFlow.

Say 'exit' or 'quit' to exit the loop
Question: exit
Exiting the conversation. Goodbye!
